## Questions: Part one

In [1]:
from amplpy import AMPL
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import math
from itertools import permutations

In [2]:
def posterior(signal, state, Prior, Likelihood, States):
    return Prior.loc[state, 0]*Likelihood.loc[state, signal]/sum([Prior.loc[state_p, 0]*Likelihood.loc[state_p, signal] for state_p in States])  

def bid(bidder, Posterior, Theta, States):
    return sum([Theta.loc[bidder, state]*Posterior[state] for state in States])

def allocate(Bids, Slots):
    ordered_bids = sorted(Bids, reverse=True)
    return [ordered_bids[bid] for bid in range(len(Slots))]

def order_theta(Theta_df, Bids):
    Theta_df['Bid'] = Bids
    Theta_df = Theta_df.sort_values(by='Bid', ascending=False)
    Theta_df = Theta_df.drop(columns=['Bid'])
    Theta_df = Theta_df.reset_index(drop=True)
    return Theta_df

def revenue(Slots, States, Posterior, Theta, Promminence):
    Promminence.append(0)
    revenue = 0
    for j in range(len(Slots)):
        inner_sum = 0
        for state in States:
            inner_sum += Posterior[state]*Theta.loc[j+1, state]
        revenue += (j+1)*inner_sum*(Promminence[j] - Promminence[j+1])
    return revenue

def gamma(Posterior, Signals, States, Prior, Likelihood):
    gamma = 0
    for signal in Signals:
        inner_sum = 0
        for state in States:
            inner_sum += Prior.loc[state, 0]*Likelihood.loc[state, signal]
        gamma += inner_sum if [posterior(signal, state_p, Prior, Likelihood, States) for state_p in States] == Posterior else 0
        
    return gamma

def check_gamma(States, Posteriors, Signals, Prior, Likelihood):
    for state in States:
        print(state)
        mu = 0
        for posterior in Posteriors:
            mu += posterior[state]*gamma(posterior, Signals, States, Prior, Likelihood) 
            print("posterior",posterior[state])
            print("gamma", gamma(posterior, Signals, States, Prior, Likelihood))
        if mu != Prior.loc[state, 0]:
            print("sum:", mu)
            print("prior:", Prior.loc[state, 0])
            raise ValueError(f'Gamma not good in state {state}!')
        
def expected_revenue(States, Bidders, Slots, Theta_df, Promminence, Posteriors, Gamma):
    # Check posterior is correct
    if np.any([sum(posterior) != 1 for posterior in Posteriors]):
        raise ValueError('Posterior must sum to 1!')

    # Check Gamma is correct
    if (sum(Gamma) != 1):
        raise ValueError('Gamma must sum to 1!')
    
    # Posteriors = [[posterior(signal, state, Prior_df, Phi_df, States) for state in States] for signal in Signals]
    expected_revenue = 0

    # Test
    # check_gamma(States, Posteriors, Signals, Prior_df, Phi_df)
    
    for i, Posterior in enumerate(Posteriors):
        Bids = [bid(bidder, Posterior, Theta_df, States) for bidder in Bidders]
        Theta_df_ordered = order_theta(Theta_df, Bids)
        rev = revenue(Slots, States, Posterior, Theta_df_ordered, Promminence)
        expected_revenue += Gamma[i]*rev
    return expected_revenue

In [3]:
def P(Bidders, Slots):
    res = list(permutations(Bidders, len(Slots)))
    res = [list(t) for t in res]
    return res

def build_v_df(Permutations, Theta_df):
    v = []
    for permutation in Permutations:
        temp = Theta_df.loc[permutation, :]
        temp = temp.reset_index(drop=True)
        v.append(temp)
    index = [i for i in range(len(Permutations))]
    return pd.concat(v, keys = index)

def compute_optimal_signaling_scheme(States, Bidders, Slots, Theta_df, Promminence, Prior):    
    Slots_plus = Slots.copy()
    Slots_plus.append(len(Slots))
    Permutations = P(Bidders, Slots_plus)
    index_permutations = [i for i in range(len(Permutations))]
    v_df = build_v_df(Permutations, Theta_df)
    
    
    Promminence.append(0)
    m = len(Slots)
    
    ampl = AMPL()
    ampl.read('model_KV.mod')
    
    ampl.set["STATES"] = States
    ampl.set["PI"] = index_permutations
    
    ampl.param["m"] = m
    ampl.param["v"] = v_df
    ampl.param["lambda"] = Promminence
    ampl.param["mu"] = Prior
    
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    
    obj = ampl.getObjective('obj').value()
    x = ampl.getVariable('x').getValues().toPandas()
    ampl.close()
    
    return obj, x

def print_result_ampl(x, obj):
    print("\nResult:")
    print("Objective value:", obj)
    # print("X:", x)
    
    pers = x.index.get_level_values(0).unique().values
    states = x.index.get_level_values(1).unique().values

    # Gamma
    gamma  = []
    for pi in pers:
        # print("Permutation", Permutations[pi])
        sum = 0
        for state in states:
            sum += x.loc[pi, state].iloc[0]
        gamma.append(sum)
        
    # print("Gamma:", gamma)

    # Posteriors
    posteriors = []
    for pi in pers:
        if gamma[pi] == 0:
            continue
        
        posterior = []
        for state in states:
            posterior.append(x.loc[pi, state].iloc[0]/gamma[pi])
        posteriors.append(posterior)

    print("\nPosteriors:")
    for i in range(len(posteriors)):
        print("Posterior", i, ":", posteriors[i])

**1. Prove that reporting the expected valuations in the posterior s is
 a dominant strategy for each bidder.**

 Since the auctioneer uses VCG and it is a Grove mechanism this means that it is a dominand strategy for the bidder give their tru expected types (valuations).

**2. What is the expected revenue of the auctioneer without committing to a
 signaling scheme?**

 Interpretation: Not commiting to a signaling scheme is equivalent to commiting to a signaling scheme that gives no information about the state.

In [4]:
# TODO: Expectation in general over all possible signal schemes

States = [0, 1, 2]
Prior = [1/len(States) for i in States] # uniform prior
Slots = [0]
Promminence = [1] 
Bidders = [0, 1, 2, 3]
Theta = [[0.9, 0.8, 0.5], 
        [0.1, 0.2, 0.1],
        [0.7, 0.8, 0.7],
        [0.3, 0.1, 0.4]]

Prior_df = pd.DataFrame(Prior, index=States)
Theta_df = pd.DataFrame(Theta, index=Bidders, columns=States)

# No information about the state
Posteriors = [[1/3, 1/3, 1/3]]
Gamma = [1]

# Check posterior is correct
if np.any([sum(posterior) != 1 for posterior in Posteriors]):
    raise ValueError('Posterior must sum to 1!')

# Check Gamma is correct
if (sum(Gamma) != 1):
    raise ValueError('Gamma must sum to 1!')

expected_rev = expected_revenue(States, Bidders, Slots, Theta_df, Promminence, Posteriors, Gamma)
print("Expected revenue:", expected_rev)

Expected revenue: 0.7333333333333333


**3. Write the LP for computing a revenue-maximizing signalings cheme and
 solve it by means of AMPL.**

In [5]:
States = [0, 1, 2]
Prior = [1/len(States) for i in States] # uniform prior
Slots = [0]
Promminence = [1] 
Bidders = [0, 1, 2, 3]
Theta = [[0.9, 0.8, 0.5], 
        [0.1, 0.2, 0.1],
        [0.7, 0.8, 0.7],
        [0.3, 0.1, 0.4]]

Prior_df = pd.DataFrame(Prior, index=States)
Theta_df = pd.DataFrame(Theta, index=Bidders, columns=States)

obj, x = compute_optimal_signaling_scheme(States, Bidders, Slots, Theta_df, Promminence, Prior)

print("Result:")
print("Objective value:", obj)
print("X:", x)

Gurobi 11.0.1:Gurobi 11.0.1: optimal solution; objective 0.7333333333
0 simplex iterations
Result:
Objective value: 0.7333333333333334
X:                   x.val
index0 index1          
0      0       0.000000
       1       0.000000
       2       0.000000
1      0       0.333333
       1       0.333333
       2       0.333333
2      0       0.000000
       1       0.000000
       2       0.000000
3      0       0.000000
       1       0.000000
       2       0.000000
4      0       0.000000
       1       0.000000
       2       0.000000
5      0       0.000000
       1       0.000000
       2       0.000000
6      0       0.000000
       1       0.000000
       2       0.000000
7      0       0.000000
       1       0.000000
       2       0.000000
8      0       0.000000
       1       0.000000
       2       0.000000
9      0       0.000000
       1       0.000000
       2       0.000000
10     0       0.000000
       1       0.000000
       2       0.000000
11     0       0.00000

In [6]:
pers = x.index.get_level_values(0).unique().values
states = x.index.get_level_values(1).unique().values

# Gamma
gamma  = []
for pi in pers:
    # print("Permutation", Permutations[pi])
    sum = 0
    for state in states:
        sum += x.loc[pi, state].iloc[0]
    gamma.append(sum)
    
print("Gamma:", gamma)

# Posteriors
posteriors = []
for pi in pers:
    if gamma[pi] == 0:
        continue
    
    posterior = []
    for state in states:
        posterior.append(x.loc[pi, state].iloc[0]/gamma[pi])
    posteriors.append(posterior)

print("Posteriors:", posteriors)

Gamma: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Posteriors: [[0.3333333333333333, 0.3333333333333333, 0.3333333333333335]]


The best signaling corresponds to not giving information about the state.